In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import chi2_contingency,chisquare
import pandas as pd
import copy, pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.optimizers import SGD

c:\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [2]:
df=pd.read_excel('data\\local files\\AI EU Data.xlsx')

In [3]:
df.head()

,Line of Business,BusinessClassification,InsuredState,InceptionDate,ExpiryDate,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,...,Total Layer Limit_l,General Liability Allocation_l,Coverage Premium_l,Uninsured/Under Insured Motorist Premium_l,Attachment Amount_l,Per Occurrence limit_l,Layer Aggregate_l,Terrorism_l,Aggregate_l,Auto Liability Allocation_l
0,Excess & Umbrella,NaN,NJ,2015-07-01,2016-07-01,1,Renewal,Issued,0,0,...,6000000.0,1866.00,19658.0,NaN,15000000.0,6000000.0,6000000.0,3034.0,6000000.0,17724.00
1,Excess & Umbrella,NaN,IN,2015-06-30,2016-06-30,1,New,Issued,0,0,...,10000000.0,53512.00,60500.0,NaN,1000000.0,10000000.0,10000000.0,NaN,10000000.0,6988.00
2,Excess & Umbrella,NaN,MN,2015-04-01,2016-06-01,1,Renewal,Issued,0,0,...,50000000.0,65266.62,66519.0,NaN,50000000.0,50000000.0,50000000.0,NaN,50000000.0,1252.38
3,Excess & Umbrella,NaN,OR,2015-03-25,2016-07-01,1,New,Issued,0,0,...,2000000.0,3220.00,3220.0,NaN,2000000.0,2000000.0,2000000.0,NaN,2000000.0,NaN
4,Excess & Umbrella,NaN,CA,2015-05-31,2016-05-31,1,Renewal,Issued,0,0,...,25000000.0,68120.00,68750.0,NaN,25000000.0,25000000.0,25000000.0,3437.5,25000000.0,630.00


In [4]:
#identify datatype missing values
df_csv=pd.DataFrame([df.dtypes
                             ,df.isna().sum()
                             ,df.count()
                             ,(df.isna().sum()/df.count().max())*100]
                           ,index = ['Datatype','Missing_count','Availability_count','Missing_percent']).T

In [5]:
df_csv.head()

,Datatype,Missing_count,Availability_count,Missing_percent
Line of Business,object,0,36537,0
BusinessClassification,object,11663,24874,31.9211
InsuredState,object,16,36521,0.0437912
InceptionDate,datetime64[ns],0,36537,0
ExpiryDate,datetime64[ns],0,36537,0


In [6]:
#Column Datatype
DATATYPE = {
           'Line of Business'          :        'categorical',
           'BusinessClassification'    :        'categorical',
           'InsuredState'              :        'categorical',
           'InceptionDate'             :        'datetime',
           'ExpiryDate'                :        'datetime',
           'IsAdmitted'                :        'categorical',
           'Type'                      :        'categorical',
           'status'                    :        'categorical',
           'Lead Excess-All Other_c'   :        'categorical',
           'Lead Excess-Real Estate/Property Mgmt/Office Building_c':        'categorical',
           'Lead Excess-Contracting/Construction_c'                 :        'categorical',
           'Excess-Real Estate/Property Mgmt/Office Building_c'     :        'categorical',
           'Excess-Manufacturing_c'                                 :        'categorical', 
           'Lead Excess-Retail/Wholesale_c'                         :        'categorical',
           'Lead Excess-Municipalities/Hospitals/Political_c'       :        'categorical',
           'Lead Excess-Installation/Service or Repair_c'           :        'categorical',
           'Excess-Installation/Service or Repair_c'                :        'categorical',
           'Excess-Entertainment/Recreation_c'                      :        'categorical',
           'Excess-Retail/Wholesale_c'                              :        'categorical',
           'Excess-Entertainment/Recreation_l'                      :        'numerical',
           'Excess-Municipalities/Hospitals/Political_c'            :        'categorical',
           'Lead Excess-Manufacturing_c'                            :        'categorical',
           'Excess-Manufacturing_l'                                 :        'numerical',
           'Excess-Contracting/Construction_c'                      :        'categorical',
           'Lead Excess-All Other_l'                                :        'numerical',
           'Lead Excess-Real Estate/Property Mgmt/Office Building_l':        'numerical',
           'Excess-All Other_c'                                     :        'categorical',
           'Umbrella-Contracting/Construction'                      :        'categorical',
           'Lead Excess-Entertainment/Recreation_c'                 :        'categorical',
           'Excess-Contracting/Construction_l'                      :        'numerical',
           'Excess-Installation/Service or Repair_l'                :        'numerical',
           'Lead Excess-Installation/Service or Repair_l'           :        'numerical',
           'Excess-Real Estate/Property Mgmt/Office Building_l'     :        'numerical',
           'Excess-All Other_l'                                     :        'numerical', 
           'Excess-Retail/Wholesale_l'                              :        'numerical', 
           'Umbrella-All Other'                                     :        'categorical',
           'Lead Excess-Contracting/Construction_l'                 :        'numerical',
           'Lead Excess-Entertainment/Recreation_l'                 :        'numerical',
           'Lead Excess-Municipalities/Hospitals/Political_l'       :        'numerical',
           'Lead Excess-Manufacturing_l'                            :        'numerical',
           'Excess-Municipalities/Hospitals/Political_l'            :        'numerical',
           'Lead Excess-Retail/Wholesale_l'                         :        'numerical', 
           'Total Layer Limit_c'                                    :        'categorical',
           'General Liability Allocation_c'                         :        'categorical', 
           'Coverage Premium_c'                                     :        'categorical',
           'Uninsured/Under Insured Motorist Premium_c'             :        'categorical', 
           'Attachment Amount_c'                                    :        'categorical',
           'Per Occurrence limit_c'                                 :        'categorical', 
           'Layer Aggregate_c'                                      :        'categorical', 
           'Terrorism_c'                                            :        'categorical', 
           'Aggregate_c'                                            :        'categorical',
           'Auto Liability Allocation_c'                            :        'categorical', 
           'Total Layer Limit_l'                                    :        'numerical',
           'General Liability Allocation_l'                         :        'numerical', 
           'Coverage Premium_l'                                     :        'numerical',
           'Uninsured/Under Insured Motorist Premium_l'             :        'numerical', 
           'Attachment Amount_l'                                    :        'numerical',
           'Per Occurrence limit_l'                                 :        'numerical', 
           'Layer Aggregate_l'                                      :        'numerical', 
           'Terrorism_l'                                            :        'numerical',
           'Aggregate_l'                                            :        'numerical', 
           'Auto Liability Allocation_l'                            :        'numerical'
           }

In [7]:
#spliting datatype json data       
def split_data_on_datatypes(data_set,datatypes_json):
    data = {}
    hash_data=set(list(datatypes_json.values()))
    for dt in hash_data:        
        lst = []
        for k in datatypes_json.keys():
            if datatypes_json[k] == dt:
                lst.append(k)
        data[dt] = lst
    for key in data.keys():        
        data[key] = data_set[data[key]]
        
    return data

In [8]:
datatypes_json_data=DATATYPE

split_data = split_data_on_datatypes(df,datatypes_json_data)
#set(datatypes_json.values()

In [9]:
def impute(x,y):
    if (x is None) or (x != x):
        return y
    else:
        return x 

In [10]:
for each in split_data['numerical']:
    y = 0
    split_data['numerical'][each] = split_data['numerical'][each].apply(lambda x: impute(x,y))
        
for each in split_data['categorical']:
    split_data['categorical'][each] = split_data['categorical'][each].apply(lambda x: impute(x,'unknown'))

features = ['Layer Aggregate_l', 'Total Layer Limit_l', 'Per Occurrence limit_l','Aggregate_l']

c:\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
# Separating out the features
df_num = split_data['numerical']
x = df_num[features]

In [12]:
 # Standardizing the features
std_scaler = StandardScaler().fit(x)

pickle.dump(std_scaler, open( r'newpcikle\\encoder\\std_scaler.pkl', "wb" ) )

In [13]:
std_scale = pickle.load( open( r'newpcikle\\encoder\\std_scaler.pkl', "rb" ) )
x = std_scale.transform(x)


In [14]:
 # PCA
    
pca = PCA(n_components=1)
principalComponents = pca.fit(x)
pickle.dump( principalComponents, open(  r'newpcikle\\encoder\\pca.pkl', "wb") )

In [15]:
principalComponent = pickle.load( open( r'newpcikle\\encoder\\pca.pkl', "rb" ) )
principalComponents = principalComponent.transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1'])

In [16]:
# merging PCA and other numerical columns
df_num.drop(features, axis=1, inplace=True)
num_data = pd.concat([principalDf,df_num],axis=1)


c:\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
y_cat =  split_data['categorical']['status']

## label encoding categorical variable
dt =pd.DataFrame()
for each in split_data['categorical'].columns:
    label_encoder = LabelEncoder()
    enc = label_encoder.fit(split_data['categorical'][each])
    pickle.dump( enc, open( r'newpcikle\\encoder\\encoder_' +  str(each.replace('/','_')) +'.pkl', "wb" ) )


In [18]:
for each in split_data['categorical'].columns:    
    encoder = pickle.load( open( r'newpcikle\\encoder\\encoder_' +  str(each.replace('/','_')) +'.pkl', "rb" ) )
    enc = encoder.transform(split_data['categorical'][each])
    dt = pd.concat([dt,pd.DataFrame(enc)],axis=1)
 

In [19]:
dt.columns = split_data['categorical'].columns
y_num = dt['status']
dt.drop('Line of Business',axis = 1,inplace = True) # removing Line of Business because there is only one Line of Business

In [20]:
dt.head()

,BusinessClassification,InsuredState,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,Lead Excess-Contracting/Construction_c,Excess-Real Estate/Property Mgmt/Office Building_c,Excess-Manufacturing_c,...,Total Layer Limit_c,General Liability Allocation_c,Coverage Premium_c,Uninsured/Under Insured Motorist Premium_c,Attachment Amount_c,Per Occurrence limit_c,Layer Aggregate_c,Terrorism_c,Aggregate_c,Auto Liability Allocation_c
0,864,33,1,1,7,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,1
1,864,17,1,0,7,0,0,0,0,0,...,1,1,1,0,1,1,1,0,1,1
2,864,25,1,1,7,0,0,0,0,0,...,2,2,2,0,2,2,2,0,2,2
3,864,39,1,0,7,0,0,0,0,0,...,2,2,2,0,2,2,2,0,2,0
4,864,6,1,1,7,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,1


In [21]:
def f1(x,y):
    return (x-y).days

In [23]:
duration = split_data['datetime'].apply(lambda x:f1(x['ExpiryDate'],x['InceptionDate']),axis=1)
cols = list(num_data.columns.values)
cols.extend(split_data['categorical'].columns.values)
cols.remove('Line of Business')
cols.append('duration')
cols.remove('status')



In [36]:
num_data.dtypes[num_data.dtypes=='object']

Series([], dtype: object)

In [50]:
df1 = pd.concat([num_data,dt,duration,y_num],axis = 1)
df1['status_cat'] = y_cat
X = df1[df1['Coverage Premium_l'] > 0]
y = X['status_cat']
X = X.drop('status',axis=1)
X = X.drop('status_cat',axis=1)
X.columns = cols

In [51]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)
temp = pd.DataFrame([X.columns.values,clf.feature_importances_],index = ['columns','imp']).T
imp_col = list(temp[temp['imp'] > 0.009]['columns'])
print(imp_col)

['principal component 1', 'General Liability Allocation_l', 'Coverage Premium_l', 'Attachment Amount_l', 'Terrorism_l', 'Auto Liability Allocation_l', 'BusinessClassification', 'InsuredState', 'duration']


In [52]:
clf = GradientBoostingClassifier()
clf.fit(X[imp_col], y)
pickle.dump( clf, open( r'newpcikle\\model\\model.pkl', "wb" ) )

In [54]:
model = pickle.load( open(r'newpcikle\\model\\model.pkl', "rb" ) )

In [55]:
y_pred_prob = pd.DataFrame(model.predict_proba(X[imp_col]),columns = model.classes_)

In [56]:
y_pred_prob

,Bound,Cancelled,Cleared,Dead,Declined,Indicated,Inquired,Issued,Outstanding Quote,Reopened,Terminated,Under Review,Void
0,0.003453,0.002445,0.001472,0.049942,0.000057,0.000047,0.007500,0.920619,0.013882,1.220472e-06,0.000580,1.443257e-06,1.346370e-06
1,0.001710,0.003848,0.001679,0.013351,0.000050,0.000028,0.000491,0.977864,0.000302,9.623468e-07,0.000674,1.201066e-06,1.059261e-06
2,0.001147,0.002993,0.000348,0.006773,0.000154,0.000012,0.000301,0.987639,0.000180,5.581793e-07,0.000452,6.962713e-07,6.140831e-07
3,0.002818,0.034653,0.002007,0.025908,0.000407,0.000053,0.001856,0.929711,0.001947,1.767674e-06,0.000637,1.162580e-08,1.944975e-06
4,0.003382,0.002071,0.001621,0.020328,0.000064,0.000046,0.001015,0.949190,0.021448,1.109537e-06,0.000832,1.398229e-06,1.238681e-06
5,0.006696,0.023600,0.000580,0.016536,0.000275,0.000044,0.000335,0.950038,0.000440,2.004193e-06,0.001445,4.844733e-06,2.314861e-06
6,0.000955,0.000770,0.000608,0.009923,0.000081,0.000013,0.000212,0.983993,0.003142,4.541468e-07,0.000301,5.585311e-07,5.000287e-07
7,0.003993,0.003729,0.001382,0.021483,0.000077,0.000032,0.001058,0.962055,0.005321,1.418391e-06,0.000866,1.760819e-06,1.559897e-06
8,0.002021,0.006703,0.001039,0.021790,0.000207,0.000032,0.001512,0.965571,0.000393,1.069614e-06,0.000730,7.726945e-09,1.176370e-06
9,0.002310,0.004823,0.002210,0.010634,0.000090,0.000031,0.000691,0.978434,0.000265,1.014200e-06,0.000509,1.164686e-06,1.114145e-06


In [57]:
model.classes_

array(['Bound', 'Cancelled', 'Cleared', 'Dead', 'Declined', 'Indicated',
       'Inquired', 'Issued', 'Outstanding Quote', 'Reopened',
       'Terminated', 'Under Review', 'Void'], dtype=object)

In [58]:
def get_score(prob):
    score = (((((prob['Bound'] + prob['Issued']+prob['Cleared']+prob['Outstanding Quote'])-(prob['Cancelled']+prob['Dead']+prob['Declined']+prob['Terminated']+prob['Void']))+1)/2)*0.9 + 0.05)*100
    return score
print('Total rows: ',len(y_pred_prob))
y_pred_prob['Score'] = None


Total rows:  36021


In [59]:
for i in range(len(y_pred_prob)):
    y_pred_prob['Score'][i] = get_score(y_pred_prob[i:i+1])[i]

c:\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
y_pred_prob

,Bound,Cancelled,Cleared,Dead,Declined,Indicated,Inquired,Issued,Outstanding Quote,Reopened,Terminated,Under Review,Void,Score
0,0.003453,0.002445,0.001472,0.049942,0.000057,0.000047,0.007500,0.920619,0.013882,1.220472e-06,0.000580,1.443257e-06,1.346370e-06,89.888
1,0.001710,0.003848,0.001679,0.013351,0.000050,0.000028,0.000491,0.977864,0.000302,9.623468e-07,0.000674,1.201066e-06,1.059261e-06,93.3634
2,0.001147,0.002993,0.000348,0.006773,0.000154,0.000012,0.000301,0.987639,0.000180,5.581793e-07,0.000452,6.962713e-07,6.140831e-07,94.0524
3,0.002818,0.034653,0.002007,0.025908,0.000407,0.000053,0.001856,0.929711,0.001947,1.767674e-06,0.000637,1.162580e-08,1.944975e-06,89.3695
4,0.003382,0.002071,0.001621,0.020328,0.000064,0.000046,0.001015,0.949190,0.021448,1.109537e-06,0.000832,1.398229e-06,1.238681e-06,92.8555
5,0.006696,0.023600,0.000580,0.016536,0.000275,0.000044,0.000335,0.950038,0.000440,2.004193e-06,0.001445,4.844733e-06,2.314861e-06,91.2153
6,0.000955,0.000770,0.000608,0.009923,0.000081,0.000013,0.000212,0.983993,0.003142,4.541468e-07,0.000301,5.585311e-07,5.000287e-07,93.993
7,0.003993,0.003729,0.001382,0.021483,0.000077,0.000032,0.001058,0.962055,0.005321,1.418391e-06,0.000866,1.760819e-06,1.559897e-06,92.5968
8,0.002021,0.006703,0.001039,0.021790,0.000207,0.000032,0.001512,0.965571,0.000393,1.069614e-06,0.000730,7.726945e-09,1.176370e-06,92.2816
9,0.002310,0.004823,0.002210,0.010634,0.000090,0.000031,0.000691,0.978434,0.000265,1.014200e-06,0.000509,1.164686e-06,1.114145e-06,93.5223


In [67]:
df_prem = pd.concat([dt[:36020],num_data[:36020],duration[:36020],y_pred_prob['Score'][:36020]],axis = 1)

In [68]:
# outlier analysis
PREDICTED_COL = 'Coverage Premium_l'

X_col =  copy.deepcopy(list(df_prem.columns.values))
X_col.remove(PREDICTED_COL)



In [69]:
df_prem.head()

,BusinessClassification,InsuredState,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,Lead Excess-Contracting/Construction_c,Excess-Real Estate/Property Mgmt/Office Building_c,Excess-Manufacturing_c,...,Excess-Municipalities/Hospitals/Political_l,Lead Excess-Retail/Wholesale_l,General Liability Allocation_l,Coverage Premium_l,Uninsured/Under Insured Motorist Premium_l,Attachment Amount_l,Terrorism_l,Auto Liability Allocation_l,0,Score
0,864,33,1,1,7,0,0,0,0,0,...,0,0,1866.00,19658.0,0.0,15000000.0,3034.0,17724.00,366,89.888
1,864,17,1,0,7,0,0,0,0,0,...,0,0,53512.00,60500.0,0.0,1000000.0,0.0,6988.00,366,93.3634
2,864,25,1,1,7,0,0,0,0,0,...,0,0,65266.62,66519.0,0.0,50000000.0,0.0,1252.38,427,94.0524
3,864,39,1,0,7,0,0,0,0,0,...,0,0,3220.00,3220.0,0.0,2000000.0,0.0,0.00,464,89.3695
4,864,6,1,1,7,0,0,0,0,0,...,0,0,68120.00,68750.0,0.0,25000000.0,3437.5,630.00,366,92.8555


In [70]:
isof = IsolationForest()
isof.fit(df_prem)
df_prem['Outlier'] = isof.predict(df_prem)



c:\python36\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [71]:
df_prem_no_outlier = df_prem[df_prem['Outlier'] == 1]
X = df_prem_no_outlier[X_col]
y = df_prem_no_outlier[PREDICTED_COL]
    

In [72]:
def train_model(X,y,epochs=2):

    input_dim = X.shape[1]

    model = Sequential()
    model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, input_dim=16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))

    # Compile model
#     sgd = SGD(lr=0.01,nesterov=True)
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X, y, nb_epoch=epochs)
    return model

In [73]:
X.head()

,BusinessClassification,InsuredState,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,Lead Excess-Contracting/Construction_c,Excess-Real Estate/Property Mgmt/Office Building_c,Excess-Manufacturing_c,...,Lead Excess-Manufacturing_l,Excess-Municipalities/Hospitals/Political_l,Lead Excess-Retail/Wholesale_l,General Liability Allocation_l,Uninsured/Under Insured Motorist Premium_l,Attachment Amount_l,Terrorism_l,Auto Liability Allocation_l,0,Score
0,864,33,1,1,7,0,0,0,0,0,...,0,0,0,1866.0,0.0,15000000.0,3034.0,17724.0,366,89.888
1,864,17,1,0,7,0,0,0,0,0,...,0,0,0,53512.0,0.0,1000000.0,0.0,6988.0,366,93.3634
4,864,6,1,1,7,0,0,0,0,0,...,0,0,0,68120.0,0.0,25000000.0,3437.5,630.0,366,92.8555
7,864,6,1,1,7,0,0,0,0,0,...,0,0,0,8496.0,0.0,10000000.0,200.0,1504.0,366,92.5968
8,864,38,1,1,7,0,0,0,0,0,...,0,0,0,4147.0,0.0,5000000.0,0.0,3353.0,366,92.2816


In [74]:
y.head()

0    19658.0
1    60500.0
4    68750.0
7    10000.0
8     7500.0
Name: Coverage Premium_l, dtype: float64

In [75]:
prem_adv_model = train_model(X,y,epochs=20)

W0917 15:59:01.166067 19172 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 15:59:01.664320 19172 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 15:59:01.766126 19172 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0917 15:59:01.864278 19172 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

c:\python36\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
W0

Epoch 1/20
32418/32418 [==============================] - 2s 65us/step - loss: 1771203936.6186
Epoch 2/20
32418/32418 [==============================] - 1s 27us/step - loss: 1077063553.8735
Epoch 3/20
32418/32418 [==============================] - 1s 27us/step - loss: 1022282395.5980
Epoch 4/20
32418/32418 [==============================] - 1s 27us/step - loss: 1138708916.3112
Epoch 5/20
32418/32418 [==============================] - 1s 27us/step - loss: 1074147199.4749
Epoch 6/20
32418/32418 [==============================] - 1s 27us/step - loss: 957971746.8910
Epoch 7/20
32418/32418 [==============================] - 1s 27us/step - loss: 973097668.0637
Epoch 8/20
32418/32418 [==============================] - 1s 27us/step - loss: 770966056.3855
Epoch 9/20
32418/32418 [==============================] - 1s 27us/step - loss: 963286443.3418
Epoch 10/20
32418/32418 [==============================] - 1s 27us/step - loss: 1041186491.5245
Epoch 11/20
32418/32418 [============================

In [76]:
model_json = prem_adv_model.to_json()

In [77]:
model_json

'{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "batch_input_shape": [null, 57], "dtype": "float32", "units": 57, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomNormal", "config": {"mean": 0.0, "stddev": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "batch_input_shape": [null, 16], "dtype": "float32", "units": 32, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomNormal", "config": {"mean": 0.0, "stddev": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_

In [78]:
with open('newpcikle\\premium_advice.json', "w" ) as json_file:
        json_file.write(model_json)
prem_adv_model.save_weights('newpcikle\\premium_advice_weights.h5')

In [79]:
json_file = open('newpcikle\\premium_advice.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


In [80]:
pa_model = model_from_json(loaded_model_json)
pa_model.load_weights('newpcikle\\premium_advice_weights.h5')
y_pred = pa_model.predict(X)    
result = pd.concat([X.reset_index(drop=True)
       ,y.reset_index(drop=True)
       ,pd.DataFrame([i[0] for i in y_pred],columns=['Predicted'])]
      ,axis=1)

In [82]:
result.head()

,BusinessClassification,InsuredState,IsAdmitted,Type,status,Lead Excess-All Other_c,Lead Excess-Real Estate/Property Mgmt/Office Building_c,Lead Excess-Contracting/Construction_c,Excess-Real Estate/Property Mgmt/Office Building_c,Excess-Manufacturing_c,...,Lead Excess-Retail/Wholesale_l,General Liability Allocation_l,Uninsured/Under Insured Motorist Premium_l,Attachment Amount_l,Terrorism_l,Auto Liability Allocation_l,0,Score,Coverage Premium_l,Predicted
0,864,33,1,1,7,0,0,0,0,0,...,0,1866.0,0.0,15000000.0,3034.0,17724.0,366,89.888,19658.0,6631.400879
1,864,17,1,0,7,0,0,0,0,0,...,0,53512.0,0.0,1000000.0,0.0,6988.0,366,93.3634,60500.0,63919.007812
2,864,6,1,1,7,0,0,0,0,0,...,0,68120.0,0.0,25000000.0,3437.5,630.0,366,92.8555,68750.0,24535.921875
3,864,6,1,1,7,0,0,0,0,0,...,0,8496.0,0.0,10000000.0,200.0,1504.0,366,92.5968,10000.0,3553.008545
4,864,38,1,1,7,0,0,0,0,0,...,0,4147.0,0.0,5000000.0,0.0,3353.0,366,92.2816,7500.0,3399.303223


In [83]:
from sklearn.metrics import mean_squared_error

In [84]:
error=mean_squared_error(result['Coverage Premium_l'], result['Predicted'])

In [85]:
error

637154888.8271693